In [1]:
cd /content/drive/MyDrive/논문/DCN

/content/drive/MyDrive/논문/DCN


In [2]:
ls

1220_dcn.ipynb           train_v2_201130.json  Wine_Meta_final_201229.csv
1227_DCN_practive.ipynb  user_test_v2.json     Wine_segment_201229.csv
test_v2_201130.json      user_train_v2.json


In [3]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 3.6MB 13.2MB/s 


In [4]:
import os
import sys
import gc
import glob
import joblib
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
import tensorflow_recommenders as tfrs

from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping, Callback
from tensorflow.keras.utils import plot_model

# 0. Data Load

In [5]:
# 0. Data Load

In [5]:
train = pd.read_json('train_v2_201130.json')
test = pd.read_json('test_v2_201130.json')

In [6]:
train.head()

,index,user_note,rating_per_user,vintage_id,user_like_count,userID,wine_id,wine_name,url,like
0,0,Soooo good 💕,4.0,164942680,0,19484511,1141133,Prestige Rosé Brut ChampagnenN.V.,/taittinger-prestige-rose-brut-champagne/w/114...,1
1,1,"Belíssimo champanhe rose, bem seco mais com mu...",4.0,164942680,2,352674,1141133,Prestige Rosé Brut ChampagnenN.V.,/taittinger-prestige-rose-brut-champagne/w/114...,1
2,4,,4.0,164942680,0,17786617,1141133,Prestige Rosé Brut ChampagnenN.V.,/taittinger-prestige-rose-brut-champagne/w/114...,1
3,5,Perfekt med gratinerede østers.,4.5,164942680,0,8078038,1141133,Prestige Rosé Brut ChampagnenN.V.,/taittinger-prestige-rose-brut-champagne/w/114...,1
4,6,Delicious!,4.0,164942680,0,3014532,1141133,Prestige Rosé Brut ChampagnenN.V.,/taittinger-prestige-rose-brut-champagne/w/114...,0


In [7]:
train.shape, test.shape

((763387, 10), (188718, 10))

In [8]:
item = pd.read_csv('Wine_segment_201229.csv')

In [9]:
item.head()

,wine_id,name,rating_count,rating_average,label_count,review_count,body,alcohol,winery_ratings_count,winery_ratings_average,winery_labels_count,winery_wines_count,Aperitif,Appetizers and snacks,Blue cheese,Cured Meat,Fruity desserts,Game,Goat cheese,Lamb,Lean fish,Mature and hard cheese,Mild and soft cheese,Mushrooms,Pasta,Pork,Poultry,Rich fish,Shellfish,Spicy food,Sweet desserts,Veal,Vegetarian,Beef,Blue cheese.1,Fruity desserts.1,Game .1,Lamb.1,Mature and hard cheese.1,Pasta.1,...,pca_18,pca_19,pca_20,pca_21,pca_22,pca_23,pca_24,pca_25,pca_26,pca_27,pca_28,pca_29,pca_30,pca_31,pca_32,pca_33,pca_34,pca_35,pca_36,pca_37,pca_38,pca_39,pca_40,pca_41,pca_42,pca_43,pca_44,pca_45,pca_46,pca_47,pca_48,pca_49,pca_50,pca_51,grapes_id,region_id,country_code,type_id,winery_id,segment
0,1938520,1882 Cabernet Sauvignon,1697,4.1,14879,16,5.0,14.5,18888.0,4.3,121618.0,62.0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,...,-1.551669,4.234245,-4.399549,-1.394831,4.230004,2.214657,3.698479,-11.544035,3.926973,2.027013,-3.108813,0.066079,3.901737,5.336387,-2.893791,-7.887779,-12.434086,5.029867,-2.870348,1.098466,0.041303,-0.516198,0.322788,-0.443685,-3.136951,0.742006,0.173241,-1.924884,-1.610956,2.868221,-2.167123,1.151749,1.444787,2.489641,[2],105.0,us,1,2412.0,2
1,14604,Les Bessards Hermitage,1078,4.3,5370,3,5.0,14.0,72079.0,3.8,462021.0,57.0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,...,3.201073,3.788378,1.210495,1.083187,0.590964,3.617076,0.106284,7.155894,1.924063,-1.217552,3.798950,-0.573204,0.910653,2.294474,-1.256222,-1.491831,-2.579453,-0.628009,-0.097134,-4.154698,-2.861205,-4.497887,1.583489,-0.026252,-1.082327,0.338037,-2.199833,-0.638129,1.981586,1.148229,-0.780446,-1.026985,-3.631833,-0.124608,[1],535.0,fr,1,7636.0,2
2,1930757,Patriarch Estate Grown,1072,4.6,6042,25,4.0,14.2,7747.0,4.4,49362.0,21.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,...,1.994470,3.356056,-0.923651,0.741282,2.596084,7.155602,-1.910814,-3.998637,-0.054258,0.587277,-0.887633,-0.478954,-0.442926,-1.749812,-1.185678,-0.141588,0.728802,-1.242658,0.493817,-1.872077,-2.067729,-3.043356,0.165190,0.615640,-0.657080,0.566004,0.658332,-0.343338,-1.285816,0.543290,-0.569400,1.647680,-1.445715,-0.359417,"[2, 10]",88.0,us,1,1905.0,2
3,1564280,Merlot,3577,4.3,18748,52,4.0,14.4,14091.0,4.4,83324.0,19.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,...,-6.055126,-6.163670,-17.412258,3.399827,0.748214,17.899010,-2.046424,-11.186761,12.007775,0.894301,5.050331,-2.478301,8.826864,7.167604,-6.359656,-1.473321,-3.462038,4.614712,-0.875028,-7.256990,-2.221329,-6.283630,-0.477432,0.779232,2.592272,-0.271975,1.884531,0.176404,4.579372,1.793655,-4.051642,3.926317,-2.261881,-0.431446,[10],24.0,us,1,1297.0,2
4,2576427,Cabernet Sauvignon F Block,115,4.4,806,1,5.0,0.0,1077.0,4.4,7749.0,18.0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0.004828,-0.478264,-1.010209,-0.003157,0.367176,-0.270334,-0.550214,0.490301,-0.315579,0.299809,-0.471115,-0.199453,-0.326484,-0.776345,-0.717146,0.147173,0.347295,-0.071307,0.385532,-0.230784,0.216109,0.222512,-0.104980,-0.133218,-0.016644,-0.063100,-0.098678,0.082621,0.164888,-0.078193,0.113466,0.270745,0.158934,-0.330067,[2],42.0,us,1,2232.0,2


# 1. Data Preprocess

In [10]:
train = train[['userID', 'wine_id', 'like']]
test = test[['userID', 'wine_id', 'like']]

In [13]:
selected_item = item[[
      'wine_id',
      'rating_count',
      'rating_average',
      'review_count',
      'body',
      'acidity_y',
      'alcohol',
      'type_id',
      'grapes_id',
      'country_code',
      'region_id',
      'segment'
]]

selected_item

,wine_id,rating_count,rating_average,review_count,body,acidity_y,alcohol,type_id,grapes_id,country_code,region_id,segment
0,1938520,1697,4.1,16,5.0,3.485010,14.5,1,[2],us,105.0,2
1,14604,1078,4.3,3,5.0,3.429150,14.0,1,[1],fr,535.0,2
2,1930757,1072,4.6,25,4.0,3.851015,14.2,1,"[2, 10]",us,88.0,2
3,1564280,3577,4.3,52,4.0,1.775668,14.4,1,[10],us,24.0,2
4,2576427,115,4.4,1,5.0,3.511364,0.0,1,[2],us,42.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
50855,1669561,788,3.5,9,3.0,3.212859,0.0,2,"[104, 34]",it,983.0,1
50856,1861275,231,3.8,6,3.0,3.072673,13.5,2,[5],it,613.0,1
50857,2201892,390,3.9,14,1.0,3.982507,0.0,3,[112],it,3232.0,1
50858,2396179,302,4.2,4,4.0,4.061171,13.5,2,[17],fr,635.0,3


In [20]:
train.loc[train['wine_id'] == 1886805, 'wine_id'] = 1183966
test.loc[test['wine_id'] == 1886805, 'wine_id'] = 1183966

In [75]:
add_train = train.merge(selected_item, on = 'wine_id', how = 'left')
add_test = test.merge(selected_item, on = 'wine_id', how = 'left')

add_train

,userID,wine_id,like,rating_count,rating_average,review_count,body,acidity_y,alcohol,type_id,grapes_id,country_code,region_id,segment
0,19484511,1141133,1,5248,4.1,1798,3.0,4.283229,12.0,3,"[5, 14, 110]",fr,409.0,3
1,352674,1141133,1,5248,4.1,1798,3.0,4.283229,12.0,3,"[5, 14, 110]",fr,409.0,3
2,17786617,1141133,1,5248,4.1,1798,3.0,4.283229,12.0,3,"[5, 14, 110]",fr,409.0,3
3,8078038,1141133,1,5248,4.1,1798,3.0,4.283229,12.0,3,"[5, 14, 110]",fr,409.0,3
4,3014532,1141133,0,5248,4.1,1798,3.0,4.283229,12.0,3,"[5, 14, 110]",fr,409.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763382,11274168,87064,0,1379,3.8,37,3.0,4.045876,12.5,2,[49],es,766.0,1
763383,11274168,63654,1,9222,3.9,178,5.0,2.339126,14.0,2,[7],fr,387.0,3
763384,11274168,5602,1,23457,4.4,865,4.0,2.695950,14.4,2,[5],us,96.0,3
763385,11274168,1396664,0,374,4.3,1,5.0,3.493441,0.0,1,[2],us,25.0,2


In [76]:
add_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 763387 entries, 0 to 763386
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   userID          763387 non-null  int64  
 1   wine_id         763387 non-null  int64  
 2   like            763387 non-null  int64  
 3   rating_count    763387 non-null  int64  
 4   rating_average  763387 non-null  float64
 5   review_count    763387 non-null  int64  
 6   body            763387 non-null  float64
 7   acidity_y       763387 non-null  float64
 8   alcohol         763387 non-null  float64
 9   type_id         763387 non-null  int64  
 10  grapes_id       763387 non-null  object 
 11  country_code    763387 non-null  object 
 12  region_id       763387 non-null  float64
 13  segment         763387 non-null  int64  
dtypes: float64(5), int64(7), object(2)
memory usage: 87.4+ MB


In [77]:
add_train.isnull().sum()

userID            0
wine_id           0
like              0
rating_count      0
rating_average    0
review_count      0
body              0
acidity_y         0
alcohol           0
type_id           0
grapes_id         0
country_code      0
region_id         0
segment           0
dtype: int64

In [78]:
add_train.head()

,userID,wine_id,like,rating_count,rating_average,review_count,body,acidity_y,alcohol,type_id,grapes_id,country_code,region_id,segment
0,19484511,1141133,1,5248,4.1,1798,3.0,4.283229,12.0,3,"[5, 14, 110]",fr,409.0,3
1,352674,1141133,1,5248,4.1,1798,3.0,4.283229,12.0,3,"[5, 14, 110]",fr,409.0,3
2,17786617,1141133,1,5248,4.1,1798,3.0,4.283229,12.0,3,"[5, 14, 110]",fr,409.0,3
3,8078038,1141133,1,5248,4.1,1798,3.0,4.283229,12.0,3,"[5, 14, 110]",fr,409.0,3
4,3014532,1141133,0,5248,4.1,1798,3.0,4.283229,12.0,3,"[5, 14, 110]",fr,409.0,3


# 2. Data Setting

In [79]:
str_features = ['userID', 'wine_id', 'type_id', 'grapes_id', 'country_code', 'region_id', 'segment']
int_features = ['rating_count', 'rating_average', 'review_count', 'body', 'acidity_y', 'alcohol', 'like']
feature_names = str_features + int_features

In [108]:
np.array(feature_names)

array(['userID', 'wine_id', 'type_id', 'grapes_id', 'country_code',
       'region_id', 'segment', 'rating_count', 'rating_average',
       'review_count', 'body', 'acidity_y', 'alcohol', 'like'],
      dtype='<U14')

In [80]:
def setType(df):

  df['rating_average'] = df['rating_average'].apply(lambda x : x * 10)
  df['acidity_y'] = df['acidity_y'].apply(lambda x : x * 10)
  df['grapes_id'] = df['grapes_id'].apply(lambda x : x[0])

  for f in int_features:
    df[f] = df[f].astype(int)

  return df

In [81]:
add_train = setType(add_train)
add_test = setType(add_test)

In [82]:
train_str_dict = {
    str_feature: [str(val).encode() for val in add_train[str_feature].values]
    for str_feature in str_features
}

train_int_dict = {
    int_feature: add_train[int_feature].values
    for int_feature in int_features
}

train_str_dict.update(train_int_dict)
train_str_dict.keys()

dict_keys(['userID', 'wine_id', 'type_id', 'grapes_id', 'country_code', 'region_id', 'segment', 'rating_count', 'rating_average', 'review_count', 'body', 'acidity_y', 'alcohol', 'like'])

In [83]:
test_str_dict = {
    str_feature: [str(val).encode() for val in add_test[str_feature].values]
    for str_feature in str_features
}

test_int_dict = {
    int_feature: add_test[int_feature].values
    for int_feature in int_features
}

test_str_dict.update(test_int_dict)
test_str_dict.keys()

dict_keys(['userID', 'wine_id', 'type_id', 'grapes_id', 'country_code', 'region_id', 'segment', 'rating_count', 'rating_average', 'review_count', 'body', 'acidity_y', 'alcohol', 'like'])

In [84]:
tensor_train = tf.data.Dataset.from_tensor_slices(train_str_dict)
tensor_test = tf.data.Dataset.from_tensor_slices(test_str_dict)

In [85]:
vocabularies = {}

for feature_name in tqdm(feature_names):
  vocab = tensor_train.batch(1_000_000).map(lambda x: x[feature_name])
  vocabularies[feature_name] = np.unique(np.concatenate(list(vocab)))

100%|██████████| 14/14 [01:12<00:00,  5.20s/it]


In [86]:
vocabularies

{'acidity_y': array([ 0,  9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
        26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42,
        43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
        60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 73, 74, 75]),
 'alcohol': array([  0,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  23,  45,  80, 112, 114,
        135]),
 'body': array([0, 1, 2, 3, 4, 5, 6, 7]),
 'country_code': array([b'al', b'am', b'ar', b'at', b'au', b'be', b'bg', b'bo', b'br',
        b'ca', b'ch', b'cl', b'cn', b'cy', b'cz', b'de', b'dk', b'es',
        b'fr', b'gb', b'ge', b'gr', b'hr', b'hu', b'il', b'in', b'it',
        b'jp', b'lb', b'lu', b'ma', b'md', b'me', b'mk', b'mt', b'mx',
        b'nl', b'nz', b'pe', b'ps', b'pt', b'ro', b'rs', b'ru', b'se',
        b'si', b'sk', b'th', b'tn', b'tr', b'ua', b'unk', b'us', b'uy',
        b'za'], dtype=object),
 'gra

# 3. Model

In [122]:
class DCN(tfrs.Model):

  def __init__(self, use_cross_layer, deep_layer_sizes, projection_dim=None):
    super().__init__()

    self.embedding_dimension = 32

    str_features = ['userID', 'wine_id', 'type_id', 'grapes_id', 'country_code', 'region_id', 'segment']
    int_features = ['rating_count', 'rating_average', 'review_count', 'body', 'acidity_y', 'alcohol']
    # feature_names = str_features + int_features

    self._all_features = str_features + int_features
    self._embeddings = {}

    # Compute embeddings for string features.
    for feature_name in str_features:
      vocabulary = vocabularies[feature_name]
      self._embeddings[feature_name] = tf.keras.Sequential(
          [tf.keras.layers.experimental.preprocessing.StringLookup(
              vocabulary=vocabulary, mask_token=None),
           tf.keras.layers.Embedding(len(vocabulary) + 1,
                                     self.embedding_dimension)
    ])
      
    # Compute embeddings for int features.
    for feature_name in int_features:
      vocabulary = vocabularies[feature_name]
      self._embeddings[feature_name] = tf.keras.Sequential(
          [tf.keras.layers.experimental.preprocessing.IntegerLookup(
              vocabulary=vocabulary, mask_value=None),
           tf.keras.layers.Embedding(len(vocabulary) + 1,
                                     self.embedding_dimension)
    ])

    if use_cross_layer:
      self._cross_layer = tfrs.layers.dcn.Cross(
          projection_dim=projection_dim,
          kernel_initializer="glorot_uniform")
    else:
      self._cross_layer = None

    self._deep_layers = [tf.keras.layers.Dense(layer_size, activation="relu")
      for layer_size in deep_layer_sizes]

    self._logit_layer = tf.keras.layers.Dense(1, activation = 'sigmoid')

    self.task = tfrs.tasks.Ranking(
      # loss=tf.keras.losses.MeanSquaredError(),
      loss = tf.keras.losses.BinaryCrossentropy(),
      metrics=[
              #  tf.keras.metrics.RootMeanSquaredError("RMSE")
              tf.keras.metrics.BinaryAccuracy(
                  name='binary_accuracy', dtype=None, threshold=0.5
                  )
               ]
    )

  def call(self, features):
    # Concatenate embeddings
    embeddings = []
    for feature_name in self._all_features:
      embedding_fn = self._embeddings[feature_name]
      embeddings.append(embedding_fn(features[feature_name]))

    x = tf.concat(embeddings, axis=1)

    # Build Cross Network
    if self._cross_layer is not None:
      x = self._cross_layer(x)
    
    # Build Deep Network
    for deep_layer in self._deep_layers:
      x = deep_layer(x)

    return self._logit_layer(x)

  def compute_loss(self, features, training=False):
    labels = features.pop("like")
    scores = self(features)
    return self.task(
        labels=labels,
        predictions=scores,
    )

In [123]:
learning_rate = 0.001

cached_train = tensor_train.shuffle(100_000).batch(8192).cache()
cached_test = tensor_test.batch(4096).cache()

In [124]:
model = DCN(use_cross_layer = True, deep_layer_sizes = [192, 192], projection_dim = None)
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate))

# 4. Train

In [125]:
history1 = model.fit(cached_train,  epochs = 10, verbose = True)

Epoch 1/10
94/94 [==============================] - 16s 150ms/step - binary_accuracy: 0.6666 - loss: 0.6068 - regularization_loss: 0.0000e+00 - total_loss: 0.6068
Epoch 2/10
94/94 [==============================] - 4s 43ms/step - binary_accuracy: 0.7333 - loss: 0.5322 - regularization_loss: 0.0000e+00 - total_loss: 0.5322
Epoch 3/10
94/94 [==============================] - 4s 44ms/step - binary_accuracy: 0.7485 - loss: 0.5081 - regularization_loss: 0.0000e+00 - total_loss: 0.5081
Epoch 4/10
94/94 [==============================] - 4s 44ms/step - binary_accuracy: 0.7594 - loss: 0.4893 - regularization_loss: 0.0000e+00 - total_loss: 0.4893
Epoch 5/10
94/94 [==============================] - 4s 44ms/step - binary_accuracy: 0.7650 - loss: 0.4784 - regularization_loss: 0.0000e+00 - total_loss: 0.4784
Epoch 6/10
94/94 [==============================] - 4s 44ms/step - binary_accuracy: 0.7708 - loss: 0.4663 - regularization_loss: 0.0000e+00 - total_loss: 0.4663
Epoch 7/10
94/94 [==============

In [114]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
import tensorflow_datasets as tfds

cached_test_numpy = tfds.as_numpy(cached_test)
y_true = [item['like'] for item in cached_test_numpy]
y_true = np.concatenate(y_true)

In [115]:
def get_result(model):
  y_pred = model.predict(cached_test).flatten()
  y_pred_class = [1 if pred > 0.5 else 0 for pred in y_pred]

  print(f"ROC: {roc_auc_score(y_true, y_pred)}")
  print(classification_report(y_true, y_pred_class))

In [126]:
get_result(model)

ROC: 0.8192714723026125
              precision    recall  f1-score   support

           0       0.72      0.71      0.71     88227
           1       0.75      0.76      0.75    100491

    accuracy                           0.74    188718
   macro avg       0.73      0.73      0.73    188718
weighted avg       0.74      0.74      0.74    188718



In [128]:
import keras
callbacks_list = [
                  keras.callbacks.EarlyStopping(
                    monitor = 'binary_accuracy',
                    patience = 15
                  )
]

In [129]:
history2 = model.fit(cached_train,
                     epochs = 500,
                     callbacks = callbacks_list,
                     verbose = True)

Epoch 1/500
94/94 [==============================] - 4s 45ms/step - binary_accuracy: 0.7860 - loss: 0.4268 - regularization_loss: 0.0000e+00 - total_loss: 0.4268
Epoch 2/500
94/94 [==============================] - 4s 45ms/step - binary_accuracy: 0.7805 - loss: 0.4305 - regularization_loss: 0.0000e+00 - total_loss: 0.4305
Epoch 3/500
94/94 [==============================] - 4s 44ms/step - binary_accuracy: 0.7886 - loss: 0.4193 - regularization_loss: 0.0000e+00 - total_loss: 0.4193
Epoch 4/500
94/94 [==============================] - 4s 44ms/step - binary_accuracy: 0.7920 - loss: 0.4104 - regularization_loss: 0.0000e+00 - total_loss: 0.4104
Epoch 5/500
94/94 [==============================] - 4s 44ms/step - binary_accuracy: 0.7880 - loss: 0.4145 - regularization_loss: 0.0000e+00 - total_loss: 0.4145
Epoch 6/500
94/94 [==============================] - 4s 44ms/step - binary_accuracy: 0.7772 - loss: 0.4280 - regularization_loss: 0.0000e+00 - total_loss: 0.4280
Epoch 7/500
94/94 [=========

In [130]:
get_result(model)

ROC: 0.9520474448469282
              precision    recall  f1-score   support

           0       0.92      0.91      0.91     88227
           1       0.92      0.93      0.92    100491

    accuracy                           0.92    188718
   macro avg       0.92      0.92      0.92    188718
weighted avg       0.92      0.92      0.92    188718



In [131]:
model.summary()

Model: "dcn_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_103 (Sequential)  (None, 32)                2112      
_________________________________________________________________
sequential_104 (Sequential)  (None, 32)                896       
_________________________________________________________________
sequential_102 (Sequential)  (None, 32)                288       
_________________________________________________________________
sequential_96 (Sequential)   (None, 32)                1792      
_________________________________________________________________
sequential_95 (Sequential)   (None, 32)                96        
_________________________________________________________________
sequential_100 (Sequential)  (None, 32)                864       
_________________________________________________________________
sequential_99 (Sequential)   (None, 32)                254688